In [ ]:
#@title Installation
!pip install timm foolbox

     |████████████████████████████████| 296kB 5.1MB/s 
     |████████████████████████████████| 1.7MB 9.2MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 163kB 21.2MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
#@title Mount Drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
#@title Imports

import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import L2PGD

import timm

from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

from tqdm import tqdm

import cv2

import matplotlib.pyplot as plt

In [ ]:
#@title Global Variables

# data_dir = "/content/drive/MyDrive/598dataset/Detector/"
# data_dir_fgsm = "/content/drive/MyDrive/598dataset/Detector_fgsm/"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 32

input_size = 224

feature_extract = False

num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#@title Utils

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('')
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            with tqdm(dataloaders[phase], unit='batch') as tepoch:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in tepoch:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        # Get model outputs and calculate loss
                        # Special case for inception because in training it has an auxiliary output. In train
                        #   mode we calculate the loss by summing the final output and the auxiliary output
                        #   but in testing we only consider the final output.
                        if is_inception and phase == 'train':
                            # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                            outputs, aux_outputs = model(inputs)
                            loss1 = criterion(outputs, labels)
                            loss2 = criterion(aux_outputs, labels)
                            loss = loss1 + 0.4*loss2
                        else:
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)

                        _, preds = torch.max(outputs, 1)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / len(dataloaders[phase].dataset)
                epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                if phase == 'val':
                    val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()


In [ ]:
#@title Model
model = models.resnet18(pretrained=False)
num_ftr = model.fc.in_features
model.fc = nn.Linear(num_ftr, num_classes)

In [ ]:
#@title Dataloader Init
data_dir = "/content/drive/MyDrive/598dataset/detector_pgd_5/"

transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['train', 'val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=0, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [ ]:
#@title resnet18 fft
model = model.to(device)

params_to_update = model.parameters()
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)
optimizer = optim.Adam(params_to_update, lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

criterion = nn.CrossEntropyLoss()
model_ft, hist = train_model(model, dataloaders, criterion, optimizer, num_epochs=num_epochs, is_inception=False)
torch.save(model.state_dict(), '/content/drive/MyDrive/598dataset/Detector0428_ResNet50_lr0.0001.pth')


  0%|          | 0/516 [00:00<?, ?batch/s]

	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.bn2.weight
	 layer

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:94: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  0%|          | 0/52 [00:00<?, ?batch/s]

train Loss: 0.1311 Acc: 0.9682


  0%|          | 0/516 [00:00<?, ?batch/s]

...

val Loss: 0.1057 Acc: 0.9697


Epoch 4/4
----------


  2%|▏         | 1/52 [00:00<00:08,  6.34batch/s]

train Loss: 0.0049 Acc: 0.9984


100%|██████████| 52/52 [00:07<00:00,  6.64batch/s]


val Loss: 0.0313 Acc: 0.9921

Training complete in 108m 32s
Best val Acc: 0.992121


In [ ]:
# model.load_state_dict(torch.load('/content/drive/MyDrive/598dataset/ViTbase_16_224_cifar10.pth'))

# Number of classes in the dataset
num_classes = 2
batch_size = 32
input_size = 224
feature_extract = False
num_epochs = 5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#@title test FGSM-L2 Epsilon=0.1

class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()

#@title Dataloader Init
transform = {
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}


data_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/VIT/FGSM/Epsilon0.1'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['val']}
class_names = image_datasets['val'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

counter = 0
correct = 0
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    pred = model(inputs).argmax(axis=1)
    # print(pred.shape, inputs.shape)
    # print(pred)
    correct +=  pred.shape[0] - pred.sum()
    counter += pred.shape[0]

print(f"clean accuracy:  {correct/counter * 100:.1f} %")

100%|██████████| 50/50 [02:52<00:00,  3.44s/batch]

clean accuracy:  0.0 %


In [ ]:
#@title test FGSM-L2 Epsilon=0.01

class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()

#@title Dataloader Init
transform = {
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}


data_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/VIT/FGSM/Epsilon0.01'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['val']}
class_names = image_datasets['val'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

counter = 0
correct = 0
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    pred = model(inputs).argmax(axis=1)
    # print(pred.shape, inputs.shape)
    # print(pred)
    correct +=  pred.shape[0] - pred.sum()
    counter += pred.shape[0]

print(f"clean accuracy:  {correct/counter * 100:.1f} %")

100%|██████████| 50/50 [02:45<00:00,  3.30s/batch]

clean accuracy:  1.2 %
